In [1]:
from model import *
from utils import *
from gen_img import gen_imgs
import datetime

Using TensorFlow backend.


In [2]:
# Create result folders 
save_path = join(get_save_path(), 'SimpleGrid')
new_path = join(save_path, 'best_val')
time = str(datetime.datetime.now()).replace(' ', '_')
workpath = join(save_path, time)
ensure_dir(new_path)
ensure_dir(workpath)
ensure_dir(save_path)


imgs = gen_imgs('nofourier')
X_train = imgs['X_train_imgs']
X_val = imgs['X_valid_imgs']
X_test = imgs['X_test_imgs']
y_train = imgs['y_train']
y_val = imgs['y_val']
y_test = imgs['y_test']

X_train = X_train.reshape((X_train.shape[0],
                           X_train.shape[1],
                           X_train.shape[2],
                           X_train.shape[3],
                           1))
X_val = X_val.reshape((X_val.shape[0],
                       X_val.shape[1],
                       X_val.shape[2],
                       X_val.shape[3],
                       1))
X_test = X_test.reshape((X_test.shape[0],
                         X_test.shape[1],
                         X_test.shape[2],
                         X_test.shape[3],
                         1))
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)


Loading data pickle...
Data pickle loaded.
(11844, 6, 7, 100, 1)
(2961, 6, 7, 100, 1)
(3101, 6, 7, 100, 1)
(11844, 4)
(2961, 4)
(3101, 4)


In [3]:
config = {
    # Network
    'num_inputs': X_train.shape[0],
    'input_shape': (X_train.shape[0], X_train.shape[1],X_train.shape[2],X_train.shape[3]),
    'epochs': 50,
    'dropout': 0.5,
    'batch_size': 320,
    'l2': 0.05,
    'lr': 0.001
}

In [4]:
simplegridmodel = SimpleGridModel()
simplegridmodel.build_model(config)
simplegridmodel.train(X_train, y_train, X_val, y_val, config, save_dir=workpath)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 4, 6, 49, 32)      800       
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 6, 49, 32)      24        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 2, 3, 49, 32)      0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 2, 3, 49, 1)       33        
_________________________________________________________________
reshape_1 (Reshape)          (None, 6, 49)             0         
_________________________________________________________________
gru_1 (GRU)                  (None, 20)                4200      
_________________________________________________________________
dense_1 (Dense)              (None, 32)               

11844/11844 [==============================] - 31s 3ms/step - loss: 1.0250 - accuracy: 0.7194 - val_loss: 1.3777 - val_accuracy: 0.3441
Epoch 46/50
11844/11844 [==============================] - 31s 3ms/step - loss: 1.0034 - accuracy: 0.7414 - val_loss: 1.3698 - val_accuracy: 0.3576
Epoch 47/50
11844/11844 [==============================] - 31s 3ms/step - loss: 0.9809 - accuracy: 0.7658 - val_loss: 1.3689 - val_accuracy: 0.3560
Epoch 48/50
11844/11844 [==============================] - 31s 3ms/step - loss: 0.9750 - accuracy: 0.7707 - val_loss: 1.3697 - val_accuracy: 0.3549
Epoch 49/50
11844/11844 [==============================] - 31s 3ms/step - loss: 0.9753 - accuracy: 0.7719 - val_loss: 1.3718 - val_accuracy: 0.3526
Epoch 50/50
11844/11844 [==============================] - 30s 3ms/step - loss: 0.9696 - accuracy: 0.7790 - val_loss: 1.3756 - val_accuracy: 0.3492


In [7]:
preds = [simplegridmodel.predict(X_test[i::443], 0) for i in range(443)]
majority_pred = []
for pred in preds:
    arg_maxes = [np.argmax(p) for p in pred]
    votes = np.bincount(arg_maxes)
    out = np.ndarray(shape=(4,), buffer=np.zeros(4), dtype=int)
    out[votes.argmax()] = 1
    majority_pred.append(out)
result = [(a == b).all() for a, b in zip(majority_pred, y_test[0:443])]
raw = simplegridmodel.evaluate(X_test, y_test)
print("Raw Acc result: {}".format(raw[1]))
print("Majority Vote result: {}".format(np.mean(result)))
# filepath = join(save_path, 'simplegridmodel.pickle')
# replace_model_if_better(filepath, np.mean(result), simplegridmodel, config)

3101/3101 [==============================] - 0s 156us/step
Raw Acc result: 0.37568527460098267
Majority Vote result: 0.3905191873589165
